In [ ]:
import requests
import csv
from datetime import datetime
import time
import random
import sys
from twilio.rest import Client
import logging

# Twilio credentials
logging.basicConfig(filename="ping_result.log", level=logging.INFO, format="%(asctime)s - %(message)s")

client = Client(account_sid, auth_token)

valid_pdf_count = 0

def send_alert(message):
    """Sends an SMS alert using Twilio."""
    try:
        client.messages.create(
            body=message,
            from_=twilio_phone_number,
            to=your_phone_number
        )
        print("Alert sent successfully.")
    except Exception as e:
        print(f"Failed to send alert: {e}")

def is_valid_pdf(check_no):
    """Checks if the PDF URL is valid. Sends an alert if 403 Forbidden or ERR999 is encountered."""
    url = f"https://ai.carinfo.co.kr/view/carinfo?check_no={check_no}"
    global valid_pdf_count
    valid_pdf_count =1 

    try:
        response = requests.get(url, timeout=10)
        # print('response.text', response.text, 'url', url)
        if response.status_code == 403 or "ERR999" in response.text:
            alert_message = f"🚨 Access denied (403 or ERR999) for check_no: {check_no}. Stopping the program."
            logging.info(f"[{valid_pdf_count}] {alert_message}")
            return -1
        
            # print(alert_message)
            # send_alert(alert_message)
            # sys.exit()  # Stop the program on a 403 error or ERR999

        #elif response.status_code == 200 and "carinfo_pdf" in response.url:
        
        elif response.status_code == 200 and ('자동차 기본정보' in response.text):
            # logging.info(f"found text: 자동차 기본정보")
            return 1
            
        elif '1년 이상' in response.text or '성능점검기록부가' in response.text:
            # logging.info(f"found text: 성능점검기록부")
            return 1
        elif '차량번호가 없습니다' in response.text:
            # logging.info(f"found text: 차량정보 없음")
            return 0
        else:
            return 0
        
    except Exception as e:
        print(f"Error occurred: {e}")
        return False
    
def binary_search_last_serial(year_code, performance_code):
    """Binary searches the last valid serial number for a given year and performance code."""
    # print(f"\n년도코드: {year_code}, 성능장코드: {performance_code} 검색 시작")
    print('start!')
    first_check_no = f"{year_code:02d}{performance_code:03d}00047"

    if not is_valid_pdf(first_check_no):
       print(f"No Data from fist serial No: {first_check_no} SKip...")
       logging.info(f"No Data from fist serial No: {first_check_no} SKip...")
       time.sleep(random.uniform(2, 10))  # Delay before moving to the next performance code
       return 0

    left, right = 1, 99999
    last_valid = 0

    while left <= right:
        mid = (left + right) // 2
        check_no = f"{year_code:02d}{performance_code:03d}{mid:05d}"

        if is_valid_pdf(check_no) == 1:
            last_valid = mid
            left = mid + 1  # Continue searching in the higher range
            logging.info(f"found valid data: {check_no}")
            print(f"found valid data: {check_no}")
        elif is_valid_pdf(check_no) == 0:
            last_valid = mid
            left = mid - 1  # Continue searching in the higher range
            logging.info(f"invalid data: {check_no}")
            print(f"invalid data: {check_no}")

        elif is_valid_pdf(check_no) == -1:
            logging.error('ERR. Awaiting another proxy server... (10 sec)')
            time.sleep(10)
            continue


        time.sleep(random.uniform(2, 10))  # Delay after each check

    print(f"final valid serials: {last_valid}")
    logging.info(f"final valid serials: {last_valid}")
    return last_valid

def search_and_save_data(start_year, end_year, start_performance, end_performance):
    """Searches for valid serials and saves data to a CSV file."""
    timestamp = datetime.now().strftime("%d%H%M")
    timestamp = datetime.now().strftime("%d%H")
    filename = f"carinfo_serial_data_{timestamp}.csv"

    print(f"CSV 파일 생성: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["URL", "년도코드", "성능장코드", "시리얼번호"])

        for year in range(start_year, end_year + 1):
            for performance in range(start_performance, end_performance + 1):
                print(f"\n년도코드: {year}, 성능장코드: {performance} 처리 시작")
                last_serial = binary_search_last_serial(year, performance)

                if last_serial > 0:
                    check_no = f"{year:02d}{performance:03d}{last_serial:05d}"
                    url = f"https://ai.carinfo.co.kr/view/carinfo_pdf?check_no={check_no}"
                    csvwriter.writerow([url, year, performance, last_serial])
                    csvfile.flush()  # Save each row immediately
                    print(f"데이터 저장 완료: 년도코드={year}, 성능장코드={performance}, 시리얼={last_serial}")
                else:
                    print(f"건너뛰기: 년도코드={year}, 성능장코드={performance} (유효한 데이터 없음)")
                    time.sleep(random.uniform(2, 10))  # Delay before moving to the next performance code

    print("\n데이터 수집 완료")

# 테스트를 위한 범위 설정
start_year, end_year = 19, 19
start_performance, end_performance = 20, 300

print(f"검색 시작: 년도 {start_year}-{end_year}, 성능장코드 {start_performance}-{end_performance}")
search_and_save_data(start_year, end_year, start_performance, end_performance)